In [1]:
%matplotlib inline
import datetime
from collections import Counter

import pandas as pd

from sentiment_classification import run_network
from utils import clean_text
from utils import calc_ratios


In [2]:
from IPython.core.display import HTML
css = open('table.css').read() + open('notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8")
reviews.head(5)

,userID,productID,userName,reviewText,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,Installing the game was a struggle (because of...,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""",If you like rally cars get this game you will ...,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""",1st shipment received a book instead of the ga...,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"I got this version instead of the PS3 version,...","awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""",I had Dirt 2 on Xbox 360 and it was an okay ga...,DIRT 3,1308009600,"06 14, 2011"


In [4]:
ratings = pd.read_csv("ratings.csv", encoding="utf-8")
ratings.head(5)

,userID,productID,ratings,timestamp
0,A24SSUT5CSW8BH,0078764343,5.0,1377302400
1,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
2,A10BECPH7W8HM7,043933702X,5.0,1404950400
3,A2PRV9OULX1TWP,043933702X,5.0,1386115200
4,AE7GUHCDQQ4UI,043933702X,1.0,1366156800


#### How many reviews are there?

In [5]:
len(reviews)

231780

In [16]:
grouped=reviews.groupby('userID')

In [18]:
grouped.head(5)

,userID,productID,userName,reviewText,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,Installing the game was a struggle (because of...,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""",If you like rally cars get this game you will ...,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""",1st shipment received a book instead of the ga...,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"I got this version instead of the PS3 version,...","awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""",I had Dirt 2 on Xbox 360 and it was an okay ga...,DIRT 3,1308009600,"06 14, 2011"
5,A2UTRVO4FDCBH6,0700099867,A.R.G.,"Overall this is a well done racing game, with ...","Good racing game, terrible Windows Live Requir...",1368230400,"05 11, 2013"
6,AN3YYDZAS3O1Y,0700099867,Bob,Loved playing Dirt 2 and I thought the graphic...,A step up from Dirt 2 and that is terrific!,1313280000,"08 14, 2011"
7,AQTC623NCESZW,0700099867,Chesty Puller,I can't tell you what a piece of dog**** this ...,Crash 3 is correct name AKA Microsoft,1353715200,"11 24, 2012"
8,A1QJJU33VNC4S7,0700099867,D@rkFX,I initially gave this one star because it was ...,A great game ruined by Microsoft's account man...,1352851200,"11 14, 2012"
9,A2JLT2WY0F2HVI,0700099867,D. Sweetapple,I still haven't figured this one out. Did ever...,Couldn't get this one to work,1391817600,"02 8, 2014"


#### Who has the most review data?

In [20]:
merged=pd.merge(reviews,ratings,on=["userID","productID"])

In [21]:
list(merged)

['userID',
 'productID',
 'userName',
 'reviewText',
 'summary',
 'unixReviewTime',
 'reviewTime',
 'ratings',
 'timestamp']

In [23]:
ratings['label'] = ratings["ratings"].apply(lambda x:'Positive' if x > 3 else 'Negative')

In [25]:
len(ratings[ratings['label'] == 'Positive'])

970029

In [ ]:
ratings_pivot = ratings.pivot(ratings,index="userID",)

In [26]:
max_id=max(reviews['userName'].value_counts())

#### Convert unixReviewTime field to date field and add it dataframe with column name date

#### What is the very first reviewText?

#### What is the name of person that made first review?

#### What is the id of video games that is the last reviewed?

#### Make summaries lowercase and remove punctuations using clean_text function

#### What is the most occurrent summary in all data?

#### What is the most occurrent summary in 2011?

#### What is the most occurent word in summaries in 2000?

#### What is the most occurent word in reviewTexts in 2000?

#### What is the most occurent word in reviewTexts before 2000?

#### Filter out reviews if there are older than 5 years

#### Join reviews and ratings dataframe on productID and userID

#### Create a column named binary_ratings and for each row fill POSITIVE if ratings is bigger than 3 else fill NEGATIVE

#### Find the most occurrent words in POSITIVE and NEGATIVE ratings

#### Calculate the ratios of counter values

#### Run network with review_ratings dataframe and fields

#### Predict whether the "good" word is positive or negative

#### Predict whether the "Bad" word is positive or negative

#### Predict "unplayable"

#### Predict "excelente"

#### Predict "excel"

#### Predict "playable"